<a href="https://colab.research.google.com/github/zelal-Eizaldeen/DLH-Project-Reproduce-HurtfulWords/blob/main/preprocessing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

------------------------------------------------------------------------------
# Names:
Zilal Eiz Al Din && Payel Chakraborty
# NetIDs:
zelalae2 && payelc2
# Purpose:
 This file contains Dataset Preprocessing
# Dataset:
 HurtfulWordsDataset (MIMIC3)
# Paper Reference:
Hurtful Words: Quantifying Biases in Clinical Contextual Word Embeddings
 - The paper: https://arxiv.org/abs/2003.11515
 - Github Repo of the paper: https://github.com/MLforHealth/HurtfulWords
 - Github Repo of the Reproduction: https://github.com/zelal-Eizaldeen/DLH-Project-Reproduce-HurtfulWords

# Usage:
Uses these downloaded MIMIC3 dataset PATIENTS, ADMISSIONS, DIAGNOSES ICD, ICUSTAYS, and NOTEEVENTS and joins them and does some processing of some of its fields.  .<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Clone DLH-Project-Reproduce-HurtfulWords
!git clone https://github.com/zelal-Eizaldeen/DLH-Project-Reproduce-HurtfulWords.git

Cloning into 'DLH-Project-Reproduce-HurtfulWords'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 278 (delta 105), reused 19 (delta 19), pack-reused 125 (from 2)
Receiving objects: 100% (278/278), 758.76 KiB | 16.14 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [ ]:
#Import modules
#Replace the PAth to your Path WHERE Constant.py located
import sys
sys.path.append('/content/DLH-Project-Reproduce-HurtfulWords/')


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import KFold

# Modules from the Github Repo of the paper
import Constants


# Upload the Pretrained Models:
 - Baseline_Clinical_BERT
 - Adversarially_Debiased_Clinical_BERT (Gender)

# Data preprocessing


# Main Steps of data preprocessing:
1. Load MIMIC3 tables.
2. Make column names uniform, Stratify patients into 12
folds by gender, Combine patient, admission, and clinical
notes, Prefer DEATHTIME over DOD, Standardize to 6
clean ethnicity categories (ex: fill missing ethnicity data
with UNKNOWN), Handle dates properly (ex: convert
DOB, CHARTDATE, DOD to datetime), Get exact age
from DOB and CHARTDATE.
3. Replace PHI placeholders with special tokens (e.g., dates → [DATE], names → [NAME], etc.)
4. Drop outpatient notes.


# Starting with analysing the dataset.

In [ ]:
# Path to the root folder of the CSVs MIMIC3 data
PATH_TO_CSVs='/path_to_MIMIC3_CSVs/mimic-iii-clinical-database-1.4/'

In [ ]:
# Load MIMIC-III tables
patients_df = pd.read_csv(f"{PATH_TO_CSVs}/PATIENTS.csv")  # Patient info
admissions_df = pd.read_csv(f"{PATH_TO_CSVs}/ADMISSIONS.csv")  # Admissions info
icds = pd.read_csv(f"{PATH_TO_CSVs}/DIAGNOSES_ICD.csv")  # Diagnoses information
icustays = pd.read_csv(f"{PATH_TO_CSVs}/ICUSTAYS.csv")  # ICU stays info/

In [ ]:
noteevents_df = pd.read_csv(f"{PATH_TO_CSVs}/NOTEEVENTS.csv")  # Notes text


<ipython-input-9-55400a7d7bf3>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents_df = pd.read_csv(f"{PATH_TO_CSVs}/NOTEEVENTS.csv")  # Notes text


In [ ]:
# noteevents_df=noteevents_df.dropna()
len(noteevents_df)

2083180

In [ ]:
noteevents_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [ ]:
noteevents_df.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT'],
      dtype='object')

In [ ]:
patients_df

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaN,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaN,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12 00:00:00,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaN,NaN,NaN,0


In [ ]:
admissions_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [ ]:
# Lowercase ICUSTAYS columns
icustays.columns=icustays.columns.str.lower()

In [ ]:
# icustays = icustays.set_index(['subject_id', 'hadm_id'])

In [ ]:
icustays

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [ ]:
# Split patients into K folds
n_splits = 12
patients_df = patients_df.sample(frac=1, random_state=42).reset_index(drop=True)
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for c, i in enumerate(kf.split(patients_df, groups=patients_df.GENDER)):
    patients_df.loc[i[1], 'fold'] = str(c)

patients_df.head()  # Show first 5 rows

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:86: UserWarning: The groups parameter is ignored by KFold
  warnings.warn(


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,fold
0,8455,8934,F,2032-12-13 00:00:00,2108-12-30 00:00:00,2108-12-30 00:00:00,2108-12-30 00:00:00,1,2
1,13889,14685,F,1845-09-04 00:00:00,2145-09-07 00:00:00,2145-09-07 00:00:00,2145-09-07 00:00:00,1,1
2,15732,16630,M,2029-02-22 00:00:00,2108-05-11 00:00:00,NaN,2108-05-11 00:00:00,1,9
3,18273,19348,M,1886-10-08 00:00:00,2186-11-01 00:00:00,NaN,2186-11-01 00:00:00,1,6
4,17506,18524,M,2092-10-18 00:00:00,2160-02-13 00:00:00,NaN,2160-02-13 00:00:00,1,0


In [ ]:
# Merge patients and admissions data
df = pd.merge(patients_df, admissions_df, on='SUBJECT_ID', how='inner')
df

,ROW_ID_x,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,fold,ROW_ID_y,...,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,8455,8934,F,2032-12-13 00:00:00,2108-12-30 00:00:00,2108-12-30 00:00:00,2108-12-30 00:00:00,1,2,10927,...,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaN,NaN,UROSEPSIS;WOUND INFECTION,1,1
1,13889,14685,F,1845-09-04 00:00:00,2145-09-07 00:00:00,2145-09-07 00:00:00,2145-09-07 00:00:00,1,1,17967,...,Medicare,NaN,JEWISH,WIDOWED,WHITE,2145-09-03 18:52:00,2145-09-04 17:39:00,PNAUMONIA,1,1
2,15732,16630,M,2029-02-22 00:00:00,2108-05-11 00:00:00,NaN,2108-05-11 00:00:00,1,9,20309,...,Medicare,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,CORONARY ARTERY DISEASE\CATH,0,1
3,18273,19348,M,1886-10-08 00:00:00,2186-11-01 00:00:00,NaN,2186-11-01 00:00:00,1,6,23653,...,Medicare,NaN,UNOBTAINABLE,NaN,WHITE,2186-10-07 23:27:00,2186-10-08 06:41:00,CERVICAL SPINE DISLOCATION,0,1
4,17506,18524,M,2092-10-18 00:00:00,2160-02-13 00:00:00,NaN,2160-02-13 00:00:00,1,0,22604,...,Private,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,CONGESTIVE HEART FAILURE\CATH,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,10724,11344,M,2149-06-03 00:00:00,NaN,NaN,NaN,0,9,13901,...,Medicaid,NaN,NOT SPECIFIED,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NEWBORN,0,1
58972,45096,94553,F,2168-08-23 00:00:00,NaN,NaN,NaN,0,3,57205,...,Private,ENGL,UNOBTAINABLE,SINGLE,WHITE,2186-10-16 08:19:00,2186-10-16 10:58:00,DIABETIC KETOACIDOSIS,0,1
58973,44272,91368,F,2104-10-27 00:00:00,2153-12-28 00:00:00,2153-12-28 00:00:00,NaN,1,11,56196,...,Private,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,NaN,NaN,HYPONATREMIA,1,1
58974,1612,1704,M,2069-08-08 00:00:00,2114-10-30 00:00:00,2114-10-30 00:00:00,NaN,1,4,2064,...,Medicaid,NaN,UNOBTAINABLE,SINGLE,WHITE,2114-10-23 20:18:00,2114-10-24 02:00:00,SEPSIS,1,1


In [ ]:
# Merge DEATHTIME and DOD
def merge_death(row):
    if not pd.isnull(row.DEATHTIME):
        return row.DEATHTIME
    else:
        return row.DOD
df['DOD_MERGED'] = df.apply(merge_death, axis=1)


In [ ]:
df.head()  # Show first 5 rows

,ROW_ID_x,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,fold,ROW_ID_y,...,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED
0,8455,8934,F,2032-12-13 00:00:00,2108-12-30 00:00:00,2108-12-30 00:00:00,2108-12-30 00:00:00,1,2,10927,...,NaN,CATHOLIC,WIDOWED,WHITE,NaN,NaN,UROSEPSIS;WOUND INFECTION,1,1,2108-12-30 03:45:00
1,13889,14685,F,1845-09-04 00:00:00,2145-09-07 00:00:00,2145-09-07 00:00:00,2145-09-07 00:00:00,1,1,17967,...,NaN,JEWISH,WIDOWED,WHITE,2145-09-03 18:52:00,2145-09-04 17:39:00,PNAUMONIA,1,1,2145-09-07 17:58:00
2,15732,16630,M,2029-02-22 00:00:00,2108-05-11 00:00:00,NaN,2108-05-11 00:00:00,1,9,20309,...,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,CORONARY ARTERY DISEASE\CATH,0,1,2108-05-11 00:00:00
3,18273,19348,M,1886-10-08 00:00:00,2186-11-01 00:00:00,NaN,2186-11-01 00:00:00,1,6,23653,...,NaN,UNOBTAINABLE,NaN,WHITE,2186-10-07 23:27:00,2186-10-08 06:41:00,CERVICAL SPINE DISLOCATION,0,1,2186-11-01 00:00:00
4,17506,18524,M,2092-10-18 00:00:00,2160-02-13 00:00:00,NaN,2160-02-13 00:00:00,1,0,22604,...,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,CONGESTIVE HEART FAILURE\CATH,0,1,2160-02-13 00:00:00


In [ ]:
# Drop outpatients (no admission data in NOTEEVENTS)
noteevents_df = noteevents_df[~(pd.isnull(noteevents_df['HADM_ID']))]
# Merge NOTEEVENTS with demographics
df = pd.merge(left=noteevents_df, right=df, on='HADM_ID', how='left')
df['LANGUAGE'].unique()

array([nan, 'ENGL', 'RUSS', 'PTUN', '*FUL', 'SPAN', 'GREE', 'MAND',
       'ITAL', 'JAPA', 'SOMA', 'KORE', 'ARAB', 'CAPE', 'ETHI', '*RUS',
       '*PER', 'PORT', 'CANT', 'CAMB', 'VIET', 'POLI', 'HIND', 'URDU',
       '*YID', 'HAIT', '*URD', 'AMER', 'PERS', '**TO', '*CDI', '*BUR',
       '*CHI', '*LEB', '*ARM', 'FREN', 'THAI', 'ALBA', '*GUJ', 'TAGA',
       '*BUL', '*DEA', '*TAM', '*PHI', '*BEN', '*PUN', '*CRE', 'TURK',
       '*IBO', '*AMH', '*ROM', '* FU', 'GERM', '* BE', 'LAOT', '*HUN',
       '**SH', '*ARA', '*KHM', '*TOI', '*FIL', '*SPA', '*MAN', '*CAN',
       '*TOY', '*BOS', '*YOR', 'SERB', '*TEL', '** T', '*LIT', '*DUT',
       'BENG', '*NEP', '*MOR', '*FAR'], dtype=object)

In [ ]:
# Fill missing ethnicity data and clean ethnicity values
df['ETHNICITY'].fillna(value='UNKNOWN/NOT SPECIFIED', inplace=True)

<ipython-input-23-732f9f229723>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ETHNICITY'].fillna(value='UNKNOWN/NOT SPECIFIED', inplace=True)


In [ ]:
# Clean ethnicity into standardized groups
def cleanField(string):
    mappings = {'HISPANIC OR LATINO': 'HISPANIC/LATINO',
                'BLACK/AFRICAN AMERICAN': 'BLACK',
                'UNABLE TO OBTAIN': 'UNKNOWN/NOT SPECIFIED',
                'PATIENT DECLINED TO ANSWER': 'UNKNOWN/NOT SPECIFIED'}
    bases = ['WHITE', 'UNKNOWN/NOT SPECIFIED', 'BLACK', 'HISPANIC/LATINO', 'OTHER', 'ASIAN']

    if string in bases:
        return string
    elif string in mappings:
        return mappings[string]
    else:
        for i in bases:
            if i in string:
                return i
        return 'OTHER'

df['ETHNICITY_TO_USE'] = df['ETHNICITY'].apply(cleanField)


In [ ]:
#Let's see the type of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851344 entries, 0 to 1851343
Data columns (total 39 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ROW_ID                int64  
 1   SUBJECT_ID_x          int64  
 2   HADM_ID               float64
 3   CHARTDATE             object 
 4   CHARTTIME             object 
 5   STORETIME             object 
 6   CATEGORY              object 
 7   DESCRIPTION           object 
 8   CGID                  float64
 9   ISERROR               float64
 10  TEXT                  object 
 11  ROW_ID_x              int64  
 12  SUBJECT_ID_y          int64  
 13  GENDER                object 
 14  DOB                   object 
 15  DOD                   object 
 16  DOD_HOSP              object 
 17  DOD_SSN               object 
 18  EXPIRE_FLAG           int64  
 19  fold                  object 
 20  ROW_ID_y              int64  
 21  ADMITTIME             object 
 22  DISCHTIME             object 
 23  DEATHTI

In [ ]:
# Handle dates properly
# Convert 'chartdate' and 'dob' to datetime type
df['CHARTDATE'] = pd.to_datetime(df['CHARTDATE'], errors='coerce')
df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce')
df['DOD'] = pd.to_datetime(df['DOD'], errors='coerce')



In [ ]:
# Ensure that chartdate is later than dob (birthdate)
df = df[df.CHARTDATE >= df.DOB]
df

,ROW_ID,SUBJECT_ID_x,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED,ETHNICITY_TO_USE
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,NaN,UNKNOWN/NOT SPECIFIED
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,...,CATHOLIC,MARRIED,WHITE,2118-06-02 08:21:00,2118-06-02 21:53:00,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,0,1,2126-10-06 00:00:00,WHITE
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,CATHOLIC,MARRIED,WHITE,NaN,NaN,TRACHEALBRONCHEAL MALACEA/SDA,0,1,2126-10-06 00:00:00,WHITE
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,...,CATHOLIC,MARRIED,WHITE,2124-07-21 10:44:00,2124-07-21 17:30:00,ASTHMA;COPD EXACERBATION,0,1,2126-10-06 00:00:00,WHITE
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,JEWISH,MARRIED,WHITE,2162-03-03 15:27:00,2162-03-03 19:53:00,S/P FALL;TELEMETRY,0,1,2162-04-04 00:00:00,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851339,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,...,PROTESTANT QUAKER,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE
1851340,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,...,PROTESTANT QUAKER,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE
1851341,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,...,PROTESTANT QUAKER,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE
1851342,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,...,PROTESTANT QUAKER,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE


# issue with the age
The issue is caused by a few rows where:

dob = 1895-01-18 (very old, but still valid in real life)

chartdate = 2195-01-18 (far future, due to MIMIC-III date shifting)

This 100+ year span causes an internal overflow when Pandas tries to compute the difference using int64 timestamps.

In [ ]:
# Compute age using raw timedelta
import numpy as np
def compute_safe_age(row):
    try:
        dob = row['DOB'].to_pydatetime()
        chart = row['CHARTDATE'].to_pydatetime()
        return (chart - dob).days / 365.24
    except Exception:
        return np.nan

df['age'] = df.apply(compute_safe_age, axis=1)
# Convert age to integer (flooring)
df['age'] = df['age'].astype(int)


In [ ]:
df

,ROW_ID,SUBJECT_ID_x,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED,ETHNICITY_TO_USE,age
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,NaN,UNKNOWN/NOT SPECIFIED,86
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,...,MARRIED,WHITE,2118-06-02 08:21:00,2118-06-02 21:53:00,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,0,1,2126-10-06 00:00:00,WHITE,81
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,MARRIED,WHITE,NaN,NaN,TRACHEALBRONCHEAL MALACEA/SDA,0,1,2126-10-06 00:00:00,WHITE,82
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,...,MARRIED,WHITE,2124-07-21 10:44:00,2124-07-21 17:30:00,ASTHMA;COPD EXACERBATION,0,1,2126-10-06 00:00:00,WHITE,87
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,MARRIED,WHITE,2162-03-03 15:27:00,2162-03-03 19:53:00,S/P FALL;TELEMETRY,0,1,2162-04-04 00:00:00,WHITE,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851339,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,...,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0
1851340,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,...,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0
1851341,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,...,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0
1851342,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,...,NaN,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0


Explanation <br>

'fold' = 'NA':
For the rows where the condition df.CATEGORY.isin([...]) is True, the code assigns the value 'NA' to the fold column.
This means that the fold column will be set to 'NA' for rows where the CATEGORY is one of the specified categorie

In [ ]:
# Exclude certain categories from fold assignment
df.loc[df.CATEGORY.isin(['Discharge summary', 'Echo', 'ECG']), 'fold'] = 'NA'
df['CATEGORY'].unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition',
       'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology',
       'Nursing/other'], dtype=object)

In [ ]:
# Merge diagnoses data into the main dataframe
icds = icds.groupby('HADM_ID').agg({'ICD9_CODE': lambda x: list(x.values)}).reset_index()
df = pd.merge(left=df, right=icds, on='HADM_ID')

df

,ROW_ID,SUBJECT_ID_x,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED,ETHNICITY_TO_USE,age,ICD9_CODE
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,NaN,UNKNOWN/NOT SPECIFIED,86,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2..."
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,2118-06-02 08:21:00,2118-06-02 21:53:00,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,0,1,2126-10-06 00:00:00,WHITE,81,"[5191, 49121, 51881, 486, 2761, 2449, 311]"
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,NaN,NaN,TRACHEALBRONCHEAL MALACEA/SDA,0,1,2126-10-06 00:00:00,WHITE,82,"[5191, 5185, 496, 2762, 45340, 5533]"
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,2124-07-21 10:44:00,2124-07-21 17:30:00,ASTHMA;COPD EXACERBATION,0,1,2126-10-06 00:00:00,WHITE,87,"[51884, 5849, 34830, 49121, 2760, 4160, 3594, ..."
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,2162-03-03 15:27:00,2162-03-03 19:53:00,S/P FALL;TELEMETRY,0,1,2162-04-04 00:00:00,WHITE,82,"[80506, 5070, 42823, 2930, 4538, E882, 4280, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851338,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,...,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]"
1851339,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,...,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]"
1851340,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,...,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]"
1851341,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,...,WHITE,NaN,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]"


In [ ]:
# Map language codes to human-readable categories
def map_lang(x):
    if x == 'ENGL':
        return 'English'
    if pd.isnull(x):
        return 'Missing'
    return 'Other'
df['LANGUAGE_TO_USE'] = df['LANGUAGE'].apply(map_lang)


In [ ]:
#Convert Columns into lower letters
df.columns = df.columns.str.lower()

In [ ]:
# Check that all required columns from Constants are present in the dataframe
for i in Constants.groups:
    assert(i['name'] in df.columns), i['name']

In [ ]:
'''
In Hurtful Words:

When they do in-hospital mortality prediction, they only want notes during a hospital/ICU stay.

They need to attach notes to ICU stays to restrict analysis to a proper clinical context.

Without matching, notes might come from outpatient settings or before ICU admission — that's messy!
'''
def fill_icustay(row):
    try:
        opts = icustays.xs((row['subject_id'], row['hadm_id']), drop_level=False)
    except KeyError:  # Handle cases where the index is not found
        return None

    # Determine the charttime
    if pd.isnull(row['charttime']):
        charttime = row['chartdate'] + pd.Timedelta(days=2)
    else:
        charttime = row['charttime']

    # Filter ICU stays based on intime
    stay = opts[opts['intime'] <= charttime].sort_values(by='intime', ascending=True)

    if len(stay) == 0:
        return None

    return stay.iloc[-1]['icustay_id']

# Apply the function to filter specific categories
df['icustay_id'] = df[df.category.isin(['Discharge summary', 'Physician', 'Nursing', 'Nursing/other'])].apply(fill_icustay, axis=1)


In [ ]:
df

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,dod_merged,ethnicity_to_use,age,icd9_code,language_to_use,icustay_id
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,NaN,UNKNOWN/NOT SPECIFIED,86,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2...",Missing,None
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2118-06-02 21:53:00,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,0,1,2126-10-06 00:00:00,WHITE,81,"[5191, 49121, 51881, 486, 2761, 2449, 311]",Missing,None
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,TRACHEALBRONCHEAL MALACEA/SDA,0,1,2126-10-06 00:00:00,WHITE,82,"[5191, 5185, 496, 2762, 45340, 5533]",Missing,None
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2124-07-21 17:30:00,ASTHMA;COPD EXACERBATION,0,1,2126-10-06 00:00:00,WHITE,87,"[51884, 5849, 34830, 49121, 2760, 4160, 3594, ...",English,None
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2162-03-03 19:53:00,S/P FALL;TELEMETRY,0,1,2162-04-04 00:00:00,WHITE,82,"[80506, 5070, 42823, 2930, 4538, E882, 4280, 4...",Other,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851338,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,...,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]",Missing,None
1851339,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,...,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]",Missing,None
1851340,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,...,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]",Missing,None
1851341,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,...,NaN,NEWBORN,0,1,NaN,WHITE,0,"[V3101, 76517, 76528, 7742, V290]",Missing,None


In [ ]:
# Fill ages over 90 with 91.4 (assuming this is to standardize)  #ned to check with Zilal
df.loc[df.age >= 90, 'age'] = 91.4
len(df)

1851343

In [ ]:
# Saving the final dataframe as a pickle for future use
df.to_pickle(f"/content/drive/MyDrive/MastersDegree/DLH/Project/HurtfulWords/Payel-DLH-related/DataFiles/sampled_files/final_raw.pkl")

In [ ]:
# Display the first few rows of the dataframe
display(df.head())

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,dod_merged,ethnicity_to_use,age,icd9_code,language_to_use,icustay_id
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,NaN,UNKNOWN/NOT SPECIFIED,86.0,"[01193, 4254, 42731, 2639, 2762, 5070, 5119, 2...",Missing,None
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2118-06-02 21:53:00,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,0,1,2126-10-06 00:00:00,WHITE,81.0,"[5191, 49121, 51881, 486, 2761, 2449, 311]",Missing,None
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,TRACHEALBRONCHEAL MALACEA/SDA,0,1,2126-10-06 00:00:00,WHITE,82.0,"[5191, 5185, 496, 2762, 45340, 5533]",Missing,None
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2124-07-21 17:30:00,ASTHMA;COPD EXACERBATION,0,1,2126-10-06 00:00:00,WHITE,87.0,"[51884, 5849, 34830, 49121, 2760, 4160, 3594, ...",English,None
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2162-03-03 19:53:00,S/P FALL;TELEMETRY,0,1,2162-04-04 00:00:00,WHITE,82.0,"[80506, 5070, 42823, 2930, 4538, E882, 4280, 4...",Other,None


In [ ]:
# Saving a smaller dataset
df.to_pickle(f"<save_the_output_of_preprocessing_dataset.ipynb>")